# Denoising by proximal solvers

We try to solve the following problem:

$$
arg \;  min_\mathbf{u} \frac{1}{2}||\mathbf{u-f}||_2^2 + \sigma J(\mathbf{u})
$$

with different choices of regularization:

- L2 on Gradient $ J(\mathbf{u}) = ||\nabla \mathbf{u}||_2^2$
- L1 on Wavelet (Orthogonal operator) $ J(\mathbf{u}) = ||\mathbf{W} \mathbf{u}||_1$
- Anisotropic TV $ J(\mathbf{u}) = ||\nabla \mathbf{u}||_1$
- Anisotropic TV + Bregman iterations $ J(\mathbf{u}) = ||\nabla \mathbf{u}||_1 - \mathbf{p}_k^T \mathbf{u}$
- Isotropic TV $ J(\mathbf{u}) = ||\nabla \mathbf{u}||_{2,1}$
- Isotropic TV + Bregman iterations $ J(\mathbf{u}) = ||\nabla \mathbf{u}||_{2,1} - \mathbf{p}_k^T \mathbf{u}$

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from scipy import datasets
from scipy.signal import filtfilt
from scipy.linalg import solve
from scipy.sparse.linalg import lsqr
from pylops.basicoperators import *
from pylops.signalprocessing import *
from pylops.utils.wavelets import *
from pylops.avo.poststack import *
from pylops.optimization.sparsity import *

from pyproximal.proximal import *
from pyproximal import ProxOperator
from pyproximal.optimization.primal import *
from pyproximal.optimization.primaldual import *
from pyproximal.optimization.bregman import *

np.random.seed(0)

Data and operators preparation

In [2]:
# Load image
img = datasets.ascent()
img = img/np.max(img)
ny, nx = img.shape

# Add noise
sigman = 0.2
n = sigman*np.max(abs(img.ravel()))*np.random.uniform(-1,1, img.shape)
#n = np.random.normal(0, sigman, img.shape)
noise_img = img + n

In [3]:
# Gradient operator
sampling = 1.
Gop = Gradient(dims=(ny, nx), sampling=sampling, edge=False, 
               kind='forward', dtype='float64')

### L2 on Gradient

In [4]:
# L2 data term
l2 = L2(b=noise_img.ravel())

# L2 regularization
sigma = 2.
thik = L2(sigma=sigma)

L = 8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1.
mu = 1. / (tau*L)

iml2 = LinearizedADMM(l2, thik, Gop, tau=tau, mu=mu, 
                      x0=np.zeros_like(img.ravel()), 
                      niter=100)[0]
iml2 = iml2.reshape(img.shape)

### L1 on Wavelet

In [5]:
# L2 data term
l2 = L2(b=noise_img.ravel())

# L1 Norm of Wavelet
sigma = .1
Qop = DWT2D(img.shape, wavelet='haar', level=4)
l1 = L1(sigma=sigma)
orth = Orthogonal(l1, Qop)

tau = 1. # eigs(Identity) as Identity is the operator of f

# This converges in one iteration
imq = ProximalGradient(l2, orth, x0=np.zeros_like(img.ravel()), tau=1., niter=1, show=True)
imq = imq.reshape(img.shape)

imq1 = ProximalGradient(l2, orth, x0=np.zeros_like(img.ravel()), tau=1., niter=5, show=True)
imq1 = imq1.reshape(img.shape)

print(np.allclose(imq, imq1))

Accelerated Proximal Gradient
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.Orthogonal.Orthogonal'>
tau = 1.0	backtrack = False	beta = 5.000000e-01
epsg = 1.0	niter = 1	tol = None
niterback = 100	acceleration = None

   Itn       x[0]          f           g       J=f+eps*g       tau
     1   3.49085e-01   8.074e+02   1.654e+03   2.462e+03   1.000e+00

Total time (s) = 0.01
---------------------------------------------------------

Accelerated Proximal Gradient
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.Orthogonal.Orthogonal'>
tau = 1.0	backtrack = False	beta = 5.000000e-01
epsg = 1.0	niter = 5	tol = None
niterback = 100	acceleration = None

   Itn       x[0]          f           g       J=f+eps*g       tau
     1   3.49085e-01   8.074e+02   1.

### Anisotropic TV

In [6]:
# L2 data term
l2 = L2(b=noise_img.ravel())

# Anisotropic TV
sigma = .1
l1 = L1(sigma=sigma)

L = 8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1.
mu = tau / L

iml1 = LinearizedADMM(l2, l1, Gop, tau=tau, mu=mu, 
                      x0=np.zeros_like(img.ravel()), 
                      niter=100, show=True)[0]
iml1 = iml1.reshape(img.shape)

Linearized-ADMM
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.L1.L1'>
Linear operator (A): <class 'pylops.basicoperators.gradient.Gradient'>
tau = 1.000000e+00	mu = 1.250000e-01	niter = 100

   Itn       x[0]          f           g       J = f + g
     1   3.83351e-02   1.736e+04   8.264e+02   1.819e+04
     2   7.58463e-02   1.406e+04   7.727e+02   1.483e+04
     3   1.11237e-01   1.156e+04   6.383e+02   1.220e+04
     4   1.43698e-01   9.595e+03   5.660e+02   1.016e+04
     5   1.72849e-01   8.015e+03   5.504e+02   8.565e+03
     6   1.98420e-01   6.746e+03   5.541e+02   7.300e+03
     7   2.20306e-01   5.730e+03   5.631e+02   6.293e+03
     8   2.38615e-01   4.920e+03   5.749e+02   5.495e+03
     9   2.53645e-01   4.275e+03   5.882e+02   4.863e+03
    10   2.65812e-01   3.762e+03   6.015e+02   4.363e+03
    11   2.75586e-01   3.354e+03   6.137e+02   3.968e+03
   

### Anisotropic TV + Bregman iterations

In [7]:
# L2 data term
sigma = 1.5
l2 = L2(b=noise_img.ravel())

# Anisotropic TV
l1 = L1()

L = 8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1.
mu = tau / L

iml1b = Bregman(l2, l1, x0=np.zeros_like(img.ravel()), A=Gop, 
                 solver=LinearizedADMM, alpha=sigma, niterouter=10, show=True,
                 **dict(tau=tau, mu=mu, niter=50))
iml1b = iml1b.reshape(img.shape)

Bregman
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.L1.L1'>
Linear operator (A): <class 'pylops.basicoperators.gradient.Gradient'>
Inner Solver: <function LinearizedADMM at 0x16811a660>
alpha = 1.500000e+00	tolf = 1.000000e-10	tolx = 1.000000e-10
niter = 10

   Itn       x[0]          f           g       J = f + g
     1   3.19919e-01   3.569e+03   2.551e+03   6.121e+03
     2   3.16910e-01   3.071e+03   3.786e+03   6.856e+03
     3   3.14483e-01   2.511e+03   5.924e+03   8.435e+03
     4   3.12194e-01   2.245e+03   7.569e+03   9.814e+03
     5   3.09848e-01   2.104e+03   8.668e+03   1.077e+04
     6   3.07416e-01   2.003e+03   9.608e+03   1.161e+04
     7   3.04913e-01   1.921e+03   1.053e+04   1.245e+04
     8   3.06308e-01   1.852e+03   1.145e+04   1.330e+04
     9   3.28754e-01   1.780e+03   1.253e+04   1.431e+04
    10   4.01893e-01   1.716e+03   1.366e+04   

### Isotropic TV (with Primal-Dual)

In [8]:
# L2 data term
l2 = L2(b=noise_img.ravel())

# Isotropic TV
sigma= 0.1
l1iso = L21(ndim=2, sigma=sigma)

# Primal-dual
L =  8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1 / np.sqrt(L)
mu = 1. / (tau*L)

iml12 = PrimalDual(l2, l1iso, Gop, tau=tau, mu=mu, theta=1., 
                   x0=np.zeros_like(img.ravel()), 
                   niter=400, show=True)
iml12 = iml12.reshape(img.shape)

Primal-dual: min_x f(Ax) + x^T z + g(x)
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.L21.L21'>
Linear operator (A): <class 'pylops.basicoperators.gradient.Gradient'>
Additional vector (z): None
tau = 0.35355339059327373		mu = 0.3535533905932738
theta = 1.00		niter = 400

   Itn       x[0]          f           g          z^x       J = f + g + z^x
     1   9.01194e-02   1.199e+04   1.515e+03   0.000e+00       1.351e+04
     2   1.63412e-01   7.059e+03   1.432e+03   0.000e+00       8.490e+03
     3   2.21451e-01   4.538e+03   1.175e+03   0.000e+00       5.713e+03
     4   2.66395e-01   3.221e+03   9.936e+02   0.000e+00       4.215e+03
     5   3.00515e-01   2.518e+03   8.961e+02   0.000e+00       3.414e+03
     6   3.25770e-01   2.133e+03   8.456e+02   0.000e+00       2.979e+03
     7   3.43842e-01   1.917e+03   8.158e+02   0.000e+00       2.733e+03
     8   3.56216e-

### Isotropic TV + Bregman iterations

In [9]:
sigma= 2.
l2 = L2(b=noise_img.ravel())

# Isotropic TV
l1iso = L21(ndim=2)

# Primal-dual
L =  8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1 / np.sqrt(L)
mu = 1. / (tau*L)

iml12b = Bregman(l2, l1iso, x0=np.zeros_like(img.ravel()), 
                  solver=PrimalDual, A=Gop, alpha=sigma, 
                  niterouter=10, show=True,
                  **dict(tau=tau, mu=mu, theta=1., niter=50))
iml12b = iml12b.reshape(img.shape)

Bregman
---------------------------------------------------------
Proximal operator (f): <class 'pyproximal.proximal.L2.L2'>
Proximal operator (g): <class 'pyproximal.proximal.L21.L21'>
Linear operator (A): <class 'pylops.basicoperators.gradient.Gradient'>
Inner Solver: <function PrimalDual at 0x16811a980>
alpha = 2.000000e+00	tolf = 1.000000e-10	tolx = 1.000000e-10
niter = 10

   Itn       x[0]          f           g       J = f + g
     1   3.30472e-01   3.176e+03   3.732e+03   6.908e+03
     2   3.36947e-01   2.979e+03   4.912e+03   7.892e+03
     3   3.43091e-01   2.691e+03   6.171e+03   8.862e+03
     4   3.48271e-01   2.373e+03   8.013e+03   1.039e+04
     5   3.52603e-01   2.196e+03   9.465e+03   1.166e+04
     6   3.56232e-01   2.088e+03   1.055e+04   1.264e+04
     7   3.59286e-01   2.008e+03   1.144e+04   1.344e+04
     8   3.61871e-01   1.943e+03   1.226e+04   1.420e+04
     9   3.64071e-01   1.877e+03   1.318e+04   1.506e+04
    10   3.65959e-01   1.809e+03   1.429e+04   1.

In [10]:
fig, axs = plt.subplots(3, 2, figsize=(14, 14))
axs[0][0].imshow(img, cmap='gray', vmin=0, vmax=1)
axs[0][0].set_title('Original')
axs[0][0].axis('off')
axs[0][0].axis('tight')
axs[0][1].imshow(noise_img, cmap='gray', vmin=0, vmax=1)
axs[0][1].set_title('Noisy')
axs[0][1].axis('off')
axs[0][1].axis('tight')
axs[1][0].imshow(iml1, cmap='gray', vmin=0, vmax=1)
axs[1][0].set_title('TVaniso')
axs[1][0].axis('off')
axs[1][0].axis('tight')
axs[1][1].imshow(iml12, cmap='gray', vmin=0, vmax=1)
axs[1][1].set_title('TViso')
axs[1][1].axis('off')
axs[1][1].axis('tight');
axs[2][0].imshow(iml1b, cmap='gray', vmin=0, vmax=1)
axs[2][0].set_title('Breg+TVaniso')
axs[2][0].axis('off')
axs[2][0].axis('tight')
axs[2][1].imshow(iml12b, cmap='gray', vmin=0, vmax=1)
axs[2][1].set_title('Breg+TViso')
axs[2][1].axis('off')
axs[2][1].axis('tight')

ix_min = 84
ix_max = 155
iy = 20
plt.figure(figsize=(9, 6))
plt.plot(img[ix_min:ix_max, iy], 'k', label='True')
plt.plot(noise_img[ix_min:ix_max, iy], 'k', lw=0.5, label='Noisy')
plt.plot(iml2[ix_min:ix_max, iy], 'b', label='L2reg')
plt.plot(iml1[ix_min:ix_max, iy], 'g', label='TVaniso')
plt.plot(iml1b[ix_min:ix_max, iy], 'm', label='Breg+TVaniso')
plt.plot(iml12[ix_min:ix_max, iy], 'r', label='TViso')
plt.plot(iml12b[ix_min:ix_max, iy], 'y', label='Breg+TViso')
plt.legend();

ix = 50
iy_min = 250
iy_max = 350
plt.figure(figsize=(9, 6))
plt.plot(img[ix, iy_min:iy_max], 'k', label='True')
plt.plot(noise_img[ix, iy_min:iy_max], 'k', lw=0.5, label='Noisy')
plt.plot(iml2[ix, iy_min:iy_max], 'b', label='L2reg')
plt.plot(iml1[ix, iy_min:iy_max], 'g', label='TVaniso')
plt.plot(iml1b[ix, iy_min:iy_max], 'm', label='Breg+TVaniso')
plt.plot(iml12[ix, iy_min:iy_max], 'r', label='TViso')
plt.plot(iml12b[ix, iy_min:iy_max], 'y', label='Breg+TViso')
plt.legend();

## Salt and Pepper noise

At this point we have considered random noise. We will deal now with salt and pepper noise (which is 0 and 1 spikes added randomly to the image.

It turns out that in this case switching from a L2 norm to a L1 norm on the data term can be used to further improve our denoising capabilities:


$$
arg \;  min_\mathbf{u} ||\mathbf{u-f}||_1 + \sigma J(\mathbf{u})
$$

In [11]:
# Load image
img = misc.ascent()
#img = 2*img/np.max(img) - 1.
img = img/np.max(img)
ny, nx = img.shape

# Add salt and pepper noise
noiseperc = 0.1

isalt = np.random.permutation(np.arange(ny*nx))[:int(noiseperc*ny*nx)]
ipepper = np.random.permutation(np.arange(ny*nx))[:int(noiseperc*ny*nx)]
noise_img = img.copy().ravel()
noise_img[isalt] = img.max()
noise_img[ipepper] = img.min()
noise_img = noise_img.reshape(ny,nx)

NameError: name 'misc' is not defined

In [ ]:
# Gradient operator
sampling = 1.
Gop = Gradient(dims=(ny, nx), sampling=sampling, edge=False, 
               kind='forward', dtype='float64')

### L2 data norm with Isotropic TV

In [ ]:
# L2 data term
l2 = L2(b=noise_img.ravel())

# L1 regularization (isotropic TV)
sigma = .2
l1iso = L21(ndim=2, sigma=sigma)

# Primal-dual
L = 8. / sampling**2 # maxeig(Grad^H Grad)
tau = .1
mu = 1. / (tau*L)

iml12_l2 = PrimalDual(l2, l1iso, Gop, tau=tau, mu=mu, theta=1., x0=np.zeros_like(noise_img).ravel(), 
                      niter=400, show=True)
iml12_l2 = iml12_l2.reshape(img.shape)

### L1 data norm with Isotropic TV

In [ ]:
# L1 data term
l1 = L1(g=noise_img.ravel())

# L1 regularization (isotropic TV)
sigma = 0.7
l1iso = L21(ndim=2, sigma=sigma)

# Primal-dual
L =  8. / sampling**2 # maxeig(Grad^H Grad)
tau = 1.
mu = 1. / (tau*L)

iml12_l1 = PrimalDual(l1, l1iso, Gop, tau=tau, mu=mu, theta=1., 
                      x0=np.zeros_like(noise_img).ravel(), #x0=noise_img.ravel(), 
                      niter=100, show=True)
iml12_l1 = iml12_l1.reshape(img.shape)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 8))
axs[0][0].imshow(img, cmap='gray', vmin=0, vmax=1)
axs[0][0].set_title('Original')
axs[0][0].axis('off')
axs[0][0].axis('tight')
axs[0][1].imshow(noise_img, cmap='gray', vmin=0, vmax=1)
axs[0][1].set_title('Noisy')
axs[0][1].axis('off')
axs[0][1].axis('tight')
axs[1][0].imshow(iml12_l2, cmap='gray', vmin=0, vmax=1)
axs[1][0].set_title('L2data + TViso')
axs[1][0].axis('off')
axs[1][0].axis('tight')
axs[1][1].imshow(iml12_l1, cmap='gray', vmin=0, vmax=1)
axs[1][1].set_title('L1data + TViso')
axs[1][1].axis('off')
axs[1][1].axis('tight');